In [86]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from selenium import webdriver
from selenium.webdriver import ActionChains
import time
import dateparser

In [17]:
# Import GeckoDriverManager module.
from webdriver_manager.firefox import GeckoDriverManager
# Install the GeckoDriverManager to run FireFox web browser.
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 1 cannot be loaded. 
Get LATEST driver version for 1 cannot be loaded. 
Driver [C:\Users\Sam\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


In [200]:
# formats to scrape, based on Bandcamp URL options
FORMATS = ['all',
           'digital',
           'vinyl',
           'cd',
           'cassette']

# search categories
#   top = best-selling
#   new = new arrivals
#   rec = artist-recommended
SEARCH_CATEGORIES = ['top','new','rec']

# scrape dataframe list
scrape_data = []

'''

### Scrape Data Format ###

scrape_data = pd.DataFrame(columns=['Release Title', 
                                    'Artist Name', 
                                    'Artist Location',
                                    'Release Date',
                                    'Release URL',
                                    'Release Genre',
                                    'Release Sub-Genre',
                                    'Number of Tracks'
                                    'Track Info',
                                    'Number of Fans',
                                    'Tags'])
                                    
### dictionary entry for track listing

track_info_entry = {
    'Track Title' : '',
    'Track Lyrics' : '',
    'Track Number' : '',
    'Track Duration': ''
}
                                    
'''


# retrieve no more than this many fans for popularity index
MAX_FANS = 1000
MAX_FAN_PAGES = int(MAX_FANS / 60)

In [202]:
# scrape bandcamp release page function
def scrape_release_page(URL,entry,output=False):
    
    entry = entry.copy()
    
    entry['Release URL'] = URL
    
    # don't add entry if it already exists in scrape dataframe
    #if URL in scrape_data['Release URL']:
    #    return
    
    print('retrieving info for ' + URL + ' ...')

    headers = {'user-agent': 'Mozilla/5.0'}
    response = requests.get(URL,headers=headers)
    
    if response.status_code == 200:

        soup = bsoup(response.text, 'lxml')
        
        # Get number of tracks
        num_tracks = int(soup.find('meta',{'property':'og:description'})['content'].split(' ')[0])
        entry['Number of Tracks'] = num_tracks

        # Get title and artist name
        title = soup.find('meta',{'name':'title'})['content'].split(', by ')
        entry['Release Title'] = title[0]
        entry['Artist Name'] = title[1]
        
        # Get artist location
        entry['Artist Location'] = soup.find('span',{'class':'location secondaryText'}).string
        
        # Get release date
        release_date = soup.find('div',{'class':'tralbumData tralbum-credits'}).contents[0].string.strip('"').strip(' ').replace('releases','').replace('released','')
        release_date = dateparser.parse(release_date)
        entry['Release Date'] = release_date
        
        # Get tags
        tags = soup.find_all('a',{'class':'tag'})
        tags_list = []
        for tag in tags:
            tags_list.append(tag.string)
        entry['Tags'] = tags_list
        
        # If no genre in entry, leave blank
        if 'Release Genre' not in entry:
            entry['Release Genre'] = 'N/A'
            
        # If no subgenre in entry, leave blank
        if 'Release Sub-Genre' not in entry:
            entry['Release Sub-Genre'] = 'N/A'            
        
        # Get track information
        entry['Track Info'] = []
        #tracks = soup.find('table',{'id':'track_table'}).findChildren('tr', recursive=False)
        tracks = soup.find('table',{'id':'track_table'}).find_all('tr',{'class':'track_row_view'})
        
        for track in tracks:
            # Track number
            track_num = int(track.find('td',{'class':'track-number-col'}).div.string.strip('.'))

            # Track title
            track_title = track.find('span',{'class':'track-title'})
            if track_title:
                track_title = track_title.string
            else:
                track_title = track.find('div',{'class':'title'}).span.string
        
            # Track duration
            track_duration = track.find('span',{'class':'time secondaryText'})
            if track_duration:
                track_duration = track_duration.string.replace('\n','').strip(' ')
            else:
                track_duration = 'N/A'  
                
            # Track lyrics
            track_lyrics = 'N/A'
            track_lyric_link = track.find('div',{'class':'info_link'}).a['href']            
            if track_lyric_link and '#lyrics' in track_lyric_link:
                lyric_tag = track.findNext('tr',{'class':'lyricsRow'})
                track_lyrics = lyric_tag.find('td',{'colspan':'4'}).div.string.strip('"').replace('\r\n','\n')            
            
            # Add track object to tracks list
            track_obj = {'Track Title': track_title, 'Track Lyrics': track_lyrics,
                        'Track Number': track_num, 'Track Duration': track_duration}
            entry['Track Info'].append(track_obj)

        
        # Popularity Index
        entry['Number of Fans'] = 'N/A'
        driver.get(URL)
        foundAllFans = False
        fan_pages_searched = 0
        while foundAllFans == False and fan_pages_searched < MAX_FAN_PAGES:
            try:
                more_thumbs = driver.find_element_by_xpath('//a[@class="more-thumbs"]')
                fan_pages_searched += 1
                if 'display: none' in more_thumbs.get_attribute("style"):
                    foundAllFans = True
            except:
                foundAllFans = True
        
        if fan_pages_searched == MAX_FAN_PAGES:
            entry['Number of Fans'] = '>' + str(MAX_FANS)
        else:
            parentElement = driver.find_element_by_xpath('//div[@class="no-writing"]')
            elementList = parentElement.find_elements_by_tag_name("a")
            entry['Number of Fans'] = len(elementList)                   
                
        if output == True:
            pprint(entry)
            print()
        
        # add entry to scrape data
        scrape_data.append(entry)
    
    pass

In [155]:
scrape_release_page('https://johnkingcave.bandcamp.com/album/devil-rides-beside',{})
scrape_release_page('https://music.sufjan.com/album/a-beginners-mind',{})

retrieving info for https://johnkingcave.bandcamp.com/album/devil-rides-beside ...
{'Artist Location': 'Albuquerque, New Mexico',
 'Artist Name': 'John King Cave',
 'Number of Fans': 8,
 'Number of Tracks': 12,
 'Release Date': datetime.datetime(2020, 5, 1, 0, 0),
 'Release Genre': 'folk',
 'Release Sub-Genre': 'N/A',
 'Release Title': 'Devil Rides Beside',
 'Release URL': 'https://johnkingcave.bandcamp.com/album/devil-rides-beside',
 'Tags': ['folk',
          'alt-country',
          'alternative',
          'americana',
          'electronica',
          'folk',
          'folk pop',
          'folk punk',
          'indie pop',
          'indie rock',
          'soul',
          'Albuquerque'],
 'Track Info': [{'Track Duration': '04:27',
                 'Track Lyrics': 'Well I got a half moon, \n'
                                 'Yes I got a half moon mistletoeing me \n'
                                 "Half moon, that's quite a half smile you got "
                             

{'Artist Location': 'New York, New York',
 'Artist Name': 'Sufjan Stevens & Angelo De Augustine',
 'Number of Fans': '>1000',
 'Number of Tracks': 14,
 'Release Date': datetime.datetime(2021, 9, 24, 0, 0),
 'Release Genre': 'folk',
 'Release Sub-Genre': 'N/A',
 'Release Title': "A Beginner's Mind",
 'Release URL': 'https://music.sufjan.com/album/a-beginners-mind',
 'Tags': ['folk', 'cinematic', 'movies', 'singer-songwriter', 'New York'],
 'Track Info': [{'Track Duration': '03:43',
                 'Track Lyrics': 'I have a memory \n'
                                 'Of a time and place where history resigned\n'
                                 'Now my apology  \n'
                                 'All the light came in to fulminate my mind\n'
                                 ' \n'
                                 'Reach out, reach out \n'
                                 'To all the ones who came before you\n'
                                 'Ponder what is right\n'
                 

In [177]:
scrape_release_page('https://lucidsins.bandcamp.com/album/cursed',{})

retrieving info for https://lucidsins.bandcamp.com/album/cursed ...
{'Artist Location': 'Scotland, UK',
 'Artist Name': 'Lucid Sins',
 'Number of Fans': '>1000',
 'Number of Tracks': 8,
 'Release Date': datetime.datetime(2021, 5, 3, 0, 0),
 'Release Genre': 'retro rock',
 'Release Sub-Genre': 'N/A',
 'Release Title': 'Cursed!',
 'Release URL': 'https://lucidsins.bandcamp.com/album/cursed',
 'Tags': ['retro rock',
          'rock',
          'doom rock',
          'folk rock',
          'harmonies',
          'heavy psych',
          'occult rock',
          'prog rock',
          'psychedelic rock',
          'stoner',
          'United Kingdom'],
 'Track Info': [{'Track Duration': '03:54',
                 'Track Lyrics': 'Learn to know the secrets of the soul\n'
                                 'As we chart a course through space and time\n'
                                 'Life beyond the body may be whole\n'
                                 'Come along and see what you can find\n'

In [221]:
scrape_search_page('https://bandcamp.com/?g=rock&s=top&p=0&gn=0&f=all&t=indie',
                   {'Release Genre':'rock','Release Sub-Genre': 'indie'})

rock top 0 all indie


In [211]:
scrape_df = pd.DataFrame(scrape_data)
scrape_df = scrape_df.drop_duplicates(subset = ["Release URL"])
scrape_df

,Release Genre,Release Sub-Genre,Release URL,Number of Tracks,Release Title,Artist Name,Artist Location,Release Date,Tags,Track Info,Number of Fans
0,rock,indie,https://wednesdayband.bandcamp.com/album/twin-...,12,Twin Plagues,Wednesday,"Asheville, North Carolina",2021-08-13,"[rock, country-gaze, indie, lo-fi, shoegaze, A...","[{'Track Title': 'Twin Plagues', 'Track Lyrics...",>1000
1,rock,indie,https://kingwoman.bandcamp.com/album/celestial...,9,Celestial Blues,King Woman,"Oakland, California",2021-07-30,"[metal, rock, doom, drone, indie, post-metal, ...","[{'Track Title': 'Celestial Blues', 'Track Lyr...",>1000
2,rock,indie,https://deltasleep.bandcamp.com/album/management,7,Management,Delta Sleep,UK,2013-05-20,"[post hardcore, rock, indie, indie rock, math ...","[{'Track Title': '16:40 AM', 'Track Lyrics': '...",>1000
3,rock,indie,https://slowcoda.bandcamp.com/album/slow-coda,11,Slow Coda,Slow Coda,"Los Angeles, California",2021-08-13,"[rock, alternative, dream pop, grunge, indie, ...","[{'Track Title': 'World of Motion', 'Track Lyr...",22
4,rock,indie,https://deafheavens.bandcamp.com/album/infinit...,9,Infinite Granite,Deafheaven,San Francisco,2021-08-20,"[alternative, metal, other, rock, indie, San F...","[{'Track Title': 'Shellstar', 'Track Lyrics': ...",>1000
5,rock,indie,https://monoofjapan.bandcamp.com/album/walking...,8,"Walking Cloud and Deep Red Sky, Flag Fluttered...",MONO,"東京都, Japan",2004-04-14,"[rock, ambient rock, experimental rock, indie,...","[{'Track Title': '16.12', 'Track Lyrics': 'N/A...",>1000
6,rock,indie,https://stevehartlett.bandcamp.com/album/308,5,308,Steve Hartlett,"Newtown, Connecticut",2021-08-17,"[acoustic, rock, indie, indie rock, pop rock, ...","[{'Track Title': 'hard to say goodbye', 'Track...",40
7,rock,indie,https://agreatbigpileofleaves.bandcamp.com/alb...,10,Pono,A Great Big Pile of Leaves,"Brooklyn, New York",2021-08-13,"[alternative, indie, pop, rock, emo, indie, in...","[{'Track Title': 'Yesterday's Clothes', 'Track...",>1000
8,rock,indie,https://sandys.bandcamp.com/album/magic-mind,9,Magic Mind,Sandy's,"San Francisco, California",2021-06-11,"[rock, california, indie, psych, san francisco...","[{'Track Title': 'Dimension IV', 'Track Lyrics...",31
9,rock,indie,https://deeperchi.bandcamp.com/album/auto-pain...,30,Auto-Pain (Deluxe),Deeper,"Chicago, Illinois",2021-09-03,"[80s, rock, angular, indie, post-punk, synth, ...","[{'Track Title': 'Esoteric', 'Track Lyrics': '...",68


In [224]:
# scrape search page function
# scrape the number of pages in the given search URL
def scrape_search_page(URL,num_pages=NUM_PAGES):

    # read parameters from link
    parameters = URL.split('?')[1:][0].split('&')
    genre = 'all'
    search_category = 'top'
    location = '0'
    formatt = 'all'
    subgenre = 'N/A'
    for parameter in parameters:
        p = parameter.split('=')
        if p[0] == 'g':
            genre = p[1]
        if p[0] == 's':
            search_category = p[1]
        if p[0] == 'gn':
            location = p[1]
        if p[0] == 'f':
            formatt = p[1]
        if p[0] == 't':
            subgenre = p[1]
        if p[0] == 'w':
            time = p[1]
    
    entry = {'Release Genre': genre,
            'Release Sub-Genre': subgenre}
    
    driver.get(URL)
    time.sleep(REFRESH_RATE)
    
    # extract URLs from link tags
    release_URLS = []
    
    for i in range(num_pages):
        # this is just to ensure that the page is loaded
        
        html = driver.page_source

        # create bsoup object
        soup = bsoup(str(html), 'lxml')

        # find album link tags on this page
        link_tags = soup.find_all('a', {'class':'item-title'})
    
        for tag in link_tags:
            tag_url = tag['href'].split('?')[0]
            if tag_url not in release_URLS:
                release_URLS.append(tag_url)
            
        #driver.find_element_by_xpath('//a[@class="more-thumbs"]')
        driver.find_element_by_xpath("//a[contains(text(), 'next')]").click()

    pprint(release_URLS)

    for release_URL in release_URLS:
        scrape_release_page(release_URL,entry)

In [222]:
###################### INPUTS ################################

# list of genres to scrape
genres = ['all','rock']

# scrape for subgenres within genres?
scrape_subgenres = True

# dictionary of subgenres to scrape, if scrape_subgenres = True
SUBGENRES = {
                'rock': ['all','indie','prog-rock','post-rock','rock-roll','psychedelic-rock'],
                'metal': [],
                'alternative': [],
                'hip-hop-rap': [],
                'experimental': [],
                'punk': [],
                'folk': [],
                'pop': [],
                'acoustic': [],
                'funk': [],
                'country': [],
                'blues': []
            }

# list of search categories to scrape
search_categories = ['top']

# list of locations to scrape
# location = 0 returns search results for all locations
locations = [0] 

# list of formats to scrape
formats = ['all']

# list of weeks to scrape
# valid weeks: [-1,0,678,677,676,675,674,673]
# today, this week, last week, 2, 3, 4, 5, 6 weeks ago
weeks = []

# number of pages to scrape
NUM_PAGES = 10 

# Refresh rate (if page isn't loading fast enough to retrieve links)
REFRESH_RATE = 5

###########################################################


In [85]:
# web scraper loop

for genre in genres:
    for category in search_categories:
        for location in locations:
            for formatt in formats:
                for week in weeks:
                    
                    scrape_URL = 'https://bandcamp.com/?g=' + genre + '&s=' + category + '&p=0' + '&gn=' + str(location) + '&f=' + formatt
                    scrape_search_page(scrape_URL)
                    
                    if scrape_subgenres and genre in SUBGENRES:
                        for subgenre in SUBGENRES[genre]:
                            subgenre_scrape_URL = scrape_URL + '&t=' + subgenre
                            scrape_search_page(subgenre_scrape_URL)

[<a class="item-title" data-bind="attr: { 'href': itemURL }, text: title, click: playMe" href="https://johncarpentermusic.bandcamp.com/album/halloween-kills-ost?from=discover-top">Halloween Kills OST</a>,
 <a class="item-title" data-bind="attr: { 'href': itemURL }, text: title, click: playMe" href="https://portrayalofguilt.bandcamp.com/album/portrayal-of-guilt-chat-pile-split?from=discover-top">portrayal of guilt / Chat Pile Split</a>,
 <a class="item-title" data-bind="attr: { 'href': itemURL }, text: title, click: playMe" href="https://tossportal.bandcamp.com/album/still-slipping-vol-1?from=discover-top">still slipping vol. 1</a>,
 <a class="item-title" data-bind="attr: { 'href': itemURL }, text: title, click: playMe" href="https://chatpile.bandcamp.com/album/portrayal-of-guilt-chat-pile-split?from=discover-top">portrayal of guilt / Chat Pile Split</a>,
 <a class="item-title" data-bind="attr: { 'href': itemURL }, text: title, click: playMe" href="https://cherubs.bandcamp.com/album/slo